In [6]:
import pandas as pd
import ast

pd.set_option('display.max_columns', None)

In [23]:
heroes_df[heroes_df['hero_id']==82]['bad_against'].tolist()

["['Axe', 'Earthshaker', 'Elder Titan', 'Ember Spirit', 'Kunkka', 'Lion', 'Mars', 'Monkey King', 'Pangolier', 'Phantom Assassin', 'Sven', 'Techies', 'Venomancer', 'Viper', 'Winter Wyvern']"]

In [9]:
heroes_df = pd.read_csv('all_heros.csv')
heroes_df

,hero_id,official_name,primary_attr,attack_type,roles,legs,name_id,pros,cons,base_strength,add_strength,lvl30_strength,base_agility,add_agility,lvl30_agility,base_intelligence,add_intelligence,lvl30_intelligence,bio,bad_against,good_against,work_well_with,link,attribute
0,1,Anti-Mage,agi,Melee,"['Carry', 'Escape', 'Nuker']",2,antimage,Extremely mobile and hard to catch.\nHigh atta...,Low stat gain.\nLow health and mana.\nReliant ...,19.0,1.6,65.4,24.0,2.8,105.2,12.0,1.8,64.2,"Anti-MageLink.\n▶️.\n .\n""They who live by the...","['Axe', 'Bane', 'Disruptor', 'Doom', 'Grimstro...","['Ancient Apparition', 'Abaddon', 'Arc Warden'...","['Bane', 'Bounty Hunter', 'Dazzle', 'Dark Seer...",https://dota2.fandom.com/wiki/Anti-Mage,Agility
1,2,Axe,str,Melee,"['Initiator', 'Durable', 'Disabler', 'Carry']",2,axe,Powerful ganker once he gets his \nGreat at co...,May have a hard time against ranged heroes in ...,25.0,2.8,106.2,20.0,2.0,78.0,18.0,1.6,64.4,"Mogul Khan, the AxeLink.\n▶️.\n .\n""Axe is all...","['Doom', 'Jakiro', 'Necrophos', 'Night Stalker...","['Abaddon', 'Anti-Mage', 'Broodmother', 'Dazzl...","['Ancient Apparition', 'Dark Seer', 'Dazzle', ...",https://dota2.fandom.com/wiki/Axe,Strength
2,3,Bane,all,Ranged,"['Support', 'Disabler', 'Nuker', 'Durable']",4,bane,Long disabling time.\nExcellent starting attri...,"No area capabilities, poor against early pushi...",23.0,2.5,95.5,23.0,2.5,95.5,23.0,2.5,95.5,"Atropos, the BaneLink.\n▶️.\n .\n""I dreamt a f...","['Abaddon', 'Broodmother', 'Jakiro', 'Luna', '...","['Faceless Void', 'Legion Commander', 'Medusa'...","['Grimstroke', 'Kunkka', 'Mirana', 'Omniknight...",https://dota2.fandom.com/wiki/Bane,Universal
3,4,Bloodseeker,agi,Melee,"['Carry', 'Disabler', 'Nuker', 'Initiator']",2,bloodseeker,Fast \nRegains health by killing enemies.\nGoo...,Lacks \nVulnerable to enemy disables.\nCounter...,24.0,2.7,102.3,22.0,3.1,111.9,17.0,2.0,75.0,"Strygwyr, the BloodseekerLink.\n▶️.\n .\n""You ...","['Abaddon', 'Dazzle', 'Ember Spirit', 'Faceles...","['Anti-Mage', 'Clinkz', 'Huskar', 'Lycan', 'Me...","['Ancient Apparition', 'Batrider', 'Bounty Hun...",https://dota2.fandom.com/wiki/Bloodseeker,Agility
4,5,Crystal Maiden,int,Ranged,"['Support', 'Disabler', 'Nuker']",2,crystalmaiden,Gives team mana regeneration.\nNumerous slows ...,Very fragile.\nHorrendous base stats (low move...,17.0,2.2,80.8,16.0,1.6,62.4,16.0,3.3,111.7,"Rylai, the Crystal MaidenLink.\n▶️.\n .\n""When...","['Bounty Hunter', 'Bristleback', 'Clockwerk', ...","['Slark', 'Phantom Assassin', 'Templar Assassi...","['Juggernaut', 'Timbersaw', 'Sven']",https://dota2.fandom.com/wiki/Crystal_Maiden,Intelligence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,129,Mars,str,Melee,"['Carry', 'Initiator', 'Disabler', 'Durable']",2,mars,Strong laner.\nDurable against \nNumerous \nHi...,Low mana pool\nVulnerable to \nRequires team c...,23.0,3.7,130.3,20.0,1.7,69.3,21.0,2.2,84.8,"Mars, the First Son of HeavenLink.\n▶️.\n .\n""...","['Axe', 'Lifestealer', 'Riki', 'Batrider', 'Fa...","['Bloodseeker', 'Juggernaut', 'Legion Commande...","['Dark Seer', 'Huskar', 'Crystal Maiden', 'Dis...",https://dota2.fandom.com/wiki/Mars,Strength
120,135,Dawnbreaker,str,Melee,"['Carry', 'Durable']",2,dawnbreaker,Very tanky.\nLow cooldown spells.\nCan turn gr...,Very low mobility.\nLoses mana very fast if sh...,26.0,3.4,124.6,14.0,1.7,63.3,20.0,2.2,83.8,"Valora, the DawnbreakerLink.\n▶️.\n .\n""Darkne...","['Viper', 'Lifestealer', 'Nyx Assassin', 'Drow...","['Chen', ""Nature's Prophet"", 'Legion Commander...","['Enigma', 'Necrophos', 'Legion Commander']",https://dota2.fandom.com/wiki/Dawnbreaker,Strength
121,136,Marci,all,Melee,"['Support', 'Carry', 'Initiator', 'Disabler', ...",2,marci,Good mobility with \nVersatile to fit any role...,"Easily countered by silences, hexes, and stuns...",23.0,3.2,115.8,20.0,2.0,78.0,19.0,1.5,62.5,MarciLink.\n▶️.\n .\n*Whistle*\n.\n\n.\nLore.\...,"['Axe', 'Bloodseeker', 'Dark Willow', '

In [17]:
hero_name_to_id = dict(zip(heroes_df.official_name.tolist(), heroes_df.hero_id.tolist()))
hero_name_to_id['Mirana']

9

In [29]:
# 1. directed counter graph
lines = []
with open('counter_rl.txt', 'w') as f:
    f.write('#source_node\tsource_class\tdest_node\tdest_class\tedge_class\n')
    for idx, row in heroes_df.iterrows():
        source_node = row.hero_id
        counter_list = ast.literal_eval(row.good_against) # if good against, point to: ->
        for dest_node in counter_list:
            line = f'{source_node}\tH1\t{hero_name_to_id[dest_node]}\tH2\tH1-H2\n'
            # f.write(line)
            lines.append(line)
    for idx, row in heroes_df.iterrows():
        dest_node = row.hero_id
        counter_list = ast.literal_eval(row.bad_against) # if bad against, point from: <-
        for source_node in counter_list:
            source_node = source_node.strip(' ')
            line = f'{hero_name_to_id[source_node]}\tH1\t{dest_node}\tH2\tH1-H2\n'
            # f.write(line)
            lines.append(line)
            
    lines = list(set(lines)) # remove duplicated lines
    for line in lines:
        f.write(line)